In [2]:
import datetime
import warnings
# warnings.filterwarnings('ignore')

import pandas as pd
from binance.spot import Spot
from binance.websocket.spot.websocket_stream import SpotWebsocketStreamClient
# WebSocket API Client
from binance.websocket.spot.websocket_api import SpotWebsocketAPIClient
from Keys import *
from MultiTrade.live_environments import LiveTradingEnv
from neuralforecast.core import NeuralForecast
from ray.rllib.policy.policy import Policy
from IPython. display import display
from pprint import pprint
from utils.utils import build_market_image, sharpe_reward, symbol_map, binanace_col_map
import json
import ast
import sqlite3 as db
from utils.rendering import LiveRenderer
import subprocess
import time
import threading

In [3]:
image=build_market_image(target_pair='ETH/USDT',time_frame='1h',axis=0)

In [4]:
datetime. datetime.now()-pd.Timedelta('1h').to_pytimedelta()

datetime.datetime(2024, 5, 30, 9, 9, 7, 809201)

In [5]:
trade_target='BTC/USDT'
api_key=binanace_api_key
api_secret=binance_api_secret

client =Spot(api_key=api_key,api_secret=api_secret,base_url='https://testnet.binance.vision')
client


In [6]:
binanace_col_map={
                    "t": "date_open",
                    "T": "date_close",
                    "s": "symbol",
                    "o": "open",
                    "c": "close",
                    "h": "high",
                    "l": "low",
                    "v": "volume",
                    "x": "is_closed",
                }

In [7]:
import pytorch_lightning
import logging


In [8]:
pytorch_lightning._logger.setLevel(0)

In [9]:
import logging
logging.getLogger('lightning').setLevel(0)

In [10]:
symbol="ETHUSDT"
history_path='Trade_history/trade.db'

conn=db.connect(history_path)

In [11]:

model=NeuralForecast.load("forecasting_model",verbose=False)


Seed set to 5


Seed set to 11
Seed set to 17
Seed set to 6


In [12]:
agent_dir='Agent/final_checkpoints/policies/default_policy'
agent= Policy.from_checkpoint(agent_dir)

In [13]:
live_env_config=dict(
                name='ETHUSDT_train',
                model=model,
                agent=agent,
                api_key=api_key,
                api_secret=api_secret,
                test_net=True,
                restore_trading=False,
                target_symbol='ETH/USDT',
                time_frame='1h',
                reward_function=sharpe_reward,
                positions = [ -.5,-.25,.25, .5], # -1 (=SHORT), 0(=OUT), +1 (=LONG)
                trading_fees = 0.01/100, # 0.01% per stock buy / sell (Binance fees)
                borrow_interest_rate= 0.0003/100, # 0.0003% per timestep (one timestep = 1h here)
                max_episode_duration=168,
                verbose=0,
                )

In [14]:
self=LiveTradingEnv(**live_env_config)
self.df
obs,info=self.reset(reset_account=False)


reset_account
{'symbol': 'ETHUSDT', 'side': 'BUY', 'type': 'MARKET', 'quoteOrderQty': 3204.6784485}


In [15]:
float(self.get_account().loc[self.quote_asset,'free'])
float(self.get_account().loc[self.base_asset,'free'])

1.6929

In [16]:

def message_handler(client, message):
    try:
        print(message)
        message_data=json.loads(message)
        k_data=message_data['k']


        # print(type(data),data)
        data=pd.DataFrame([k_data])
        
        data.columns=data.columns.map(binanace_col_map)
        data=data[[c for c in binanace_col_map.values() if c in data.columns]]
        display(data)
        data=data.drop('date_open',axis=1)
        data["date_close"]=pd.to_datetime(data["date_close"],unit='ms')
        data=data.set_index('date_close')
        data['symbol'] = self.symbol
        data['unique_id']=symbol_map[self.symbol]

        
        conn=db.connect(history_path)
        data.to_sql(f'{self.symbol}_candle_history',conn,if_exists='append',index=False)
        

        if k_data['x']:
            data=self.get_data()
            self._set_df(data)
            self._prep_forecasts()
            self._set_df(data)

            obs = self._get_obs()
            
            action,_,states=self.agent.compute_single_action(obs,explore=False)
            obs, reward, terminated, truncated, info=self.live_step(action,wait=False)

            print(obs.shape)

    except Exception as e: 
        print('bad_data',message)
        print(e)

In [17]:
def on_start(message):
        obs,info=self.reset(reset_account=True)
        start_time=datetime. datetime.now().strftime("%m-%d-%Y %H:%M")
        print (f'START trading session at {start_time}')
        action,_,states=self.agent.compute_single_action(obs,explore=False)
        obs, reward, terminated, truncated, info=self.live_step(action,wait=False)

def on_close(_,message):
        end_time=datetime. datetime.now().strftime("%m-%d-%Y %H:%M")
        print (f'FINISH trading session at {end_time}')
        self.reset_account()
        

In [18]:
# my_client = SpotWebsocketAPIClient(api_key=api_key,api_secret=api_secret,on_message=self.stream_step,stream_url='wss://testnet.binance.vision')
my_client = SpotWebsocketStreamClient(
                                    on_close=on_close,
                                    on_open=on_start,
                                    on_error=on_close,
                                    on_message=self.stream_step,
                                    stream_url='wss://testnet.binance.vision')
# renderer=LiveRenderer('Trade_history/trade.db')

reset_account
{'symbol': 'ETHUSDT', 'side': 'BUY', 'type': 'MARKET', 'quoteOrderQty': 0.7301555}
(400, -1013, 'Filter failure: NOTIONAL', {'Content-Type': 'application/json;charset=UTF-8', 'Content-Length': '47', 'Connection': 'keep-alive', 'Date': 'Thu, 30 May 2024 17:09:14 GMT', 'Server': 'nginx', 'x-mbx-uuid': '54a81a44-e2ca-47b0-acda-82ffd7cbc009', 'x-mbx-used-weight': '212', 'x-mbx-used-weight-1m': '212', 'Strict-Transport-Security': 'max-age=31536000; includeSubdomains', 'X-Frame-Options': 'SAMEORIGIN', 'X-Xss-Protection': '1; mode=block', 'X-Content-Type-Options': 'nosniff', 'Content-Security-Policy': "default-src 'self'", 'X-Content-Security-Policy': "default-src 'self'", 'X-WebKit-CSP': "default-src 'self'", 'Cache-Control': 'no-cache, no-store, must-revalidate', 'Pragma': 'no-cache', 'Expires': '0', 'X-Cache': 'Error from cloudfront', 'Via': '1.1 fa29d6aa47b7c482fa023ba4aba3882c.cloudfront.net (CloudFront)', 'X-Amz-Cf-Pop': 'LAX53-P1', 'X-Amz-Cf-Id': 'jB-Mn9YW3GlcxfPDue0M9rnI

In [19]:
class LiveTradingApp(LiveRenderer):
        def __init__(self,render_logs_dir,trading_env):
                super().__init__(render_logs_dir)
                self.trading_env=trading_env
                

        def run(self):
                my_client = SpotWebsocketStreamClient(
                        on_close=on_close,
                        on_open=on_start,
                        on_error=on_close,
                        on_message=self.trading_env.stream_step,
                        stream_url='wss://testnet.binance.vision')
                
                my_client.kline(symbol=self.trading_env.symbol,interval=self.trading_env.time_frame)
                
                super().run()

In [20]:
renderer=LiveTradingApp('Trade_history/trade.db',self)
renderer.run()

reset_account
{'symbol': 'ETHUSDT', 'side': 'BUY', 'type': 'MARKET', 'quoteOrderQty': 3202.0434835}
START trading session at 05-30-2024 10:09
 -0.5
portfoliio {'asset': 1.6887, 'fiat': 6403.963957, 'borrowed_asset': 0, 'borrowed_fiat': 0, 'interest_asset': 0, 'interest_fiat': 0, 'valuation': 12838.383793}
Trade 0->-0.5
              -0.5, so 3217.21USDT / 0.84435000 ETH,
              SELL ETH:0.84435 for 3217.21$ at 3810.28 ETHUSDT
{'symbol': 'ETHUSDT', 'side': 'SELL', 'type': 'MARKET', 'quoteOrderQty': 3217.209918}
 * Serving Flask app 'gym_trading_env.renderer'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


bad_data {"result":null,"id":1717088959536}
'k'


In [ ]:
import threading
# Subscribe to a single symbol stream
app=threading.Thread(target=renderer.run)

for i in range(7):
    # my_client = SpotWebsocketAPIClient(api_key=api_key,api_secret=api_secret,on_message=self.stream_step,stream_url='wss://testnet.binance.vision')
    my_client = SpotWebsocketStreamClient(
                                        on_close=on_close,
                                        on_open=on_start,
                                        on_error=on_close,
                                        on_message=self.stream_step,
                                        stream_url='wss://testnet.binance.vision')
    wait_time=pd.Timedelta(hours=24).total_seconds()
    my_client.kline(symbol="ETHUSDT",interval="1h")
    # app=threading.Thread(target=renderer.run,kwargs=dict(symbol="ETHUSDT",interval="1h"))
    app.start()
    print('waiting to reconnect')
    time.sleep(wait_time)



reset_account
{'symbol': 'ETHUSDT', 'side': 'BUY', 'type': 'MARKET', 'quoteOrderQty': 3202.179427}
START trading session at 05-30-2024 09:51
 -0.5
portfoliio {'asset': 1.6931, 'fiat': 6404.77215, 'borrowed_asset': 0, 'borrowed_fiat': 0, 'interest_asset': 0, 'interest_fiat': 0, 'valuation': 12810.108069999998}
Trade 0->-0.5
              -0.5, so 3202.67USDT / 0.84655000 ETH,
              SELL ETH:0.84655 for 3202.67$ at 3783.2 ETHUSDT
{'symbol': 'ETHUSDT', 'side': 'SELL', 'type': 'MARKET', 'quoteOrderQty': 3202.66796}
waiting to reconnect
 * Serving Flask app 'gym_trading_env.renderer'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


bad_data {"result":null,"id":1717087903789}
'k'
stay in position


In [ ]:
pd.Timestamp(1717013599913,unit='ms')

In [ ]:
# client.avg_price('ETHUSDT'),client.ticker_price('ETHUSDT')


In [ ]:
obs,info=self.reset(reset_account=False)
obs.shape


In [ ]:
from ray.rllib.policy.policy import Policy
from ray.rllib.algorithms.algorithm import Algorithm


In [ ]:
agent_dir='Agent/final_checkpoints/policies/default_policy'
agent= Policy.from_checkpoint(agent_dir)

# agent.export_model('Agent/final_checkpoints/pretrained_agent')


In [ ]:
# action,_,states=agent.compute_single_action(obs)
# obs, reward, terminated, truncated, info=self.live_step(action,wait=False)
# action,_,states=agent.compute_single_action(obs)


In [ ]:
obs,info=self.reset(reset_account=True)
i=0
for i in range(10):
    action,_,states=agent.compute_single_action(obs,explore=False)
    obs, reward, terminated, truncated, info=self.live_step(action,wait=False)

In [ ]:
# h=self.historical_info[0]
history_df=pd.DataFrame([self.historical_info[-1]])
history_df

In [ ]:
import sqlite3 as db
import pandas as pd


symbol='ETHUSDT'
# connect to a test DB with one three-column table titled "Demo"
conn = db.connect('Trade_history/trade.db')
cur = conn .cursor()

In [ ]:
history_df=pd.read_sql(f'select * from {symbol}_trade_history',conn)
history_df.filter(like='port')

,portfolio_valuation,portfolio_distribution_asset,portfolio_distribution_fiat,portfolio_distribution_borrowed_asset,portfolio_distribution_borrowed_fiat,portfolio_distribution_interest_asset,portfolio_distribution_interest_fiat
0,12388.553857,0.0,12388.553857,0.0,0.0,0.0,0.0
1,12788.485412,NaN,NaN,NaN,NaN,NaN,NaN
2,9591.069080,NaN,NaN,NaN,NaN,NaN,NaN
3,9591.069080,NaN,NaN,NaN,NaN,NaN,NaN
4,9591.069080,NaN,NaN,NaN,NaN,NaN,NaN
5,9591.069080,NaN,NaN,NaN,NaN,NaN,NaN
6,9591.069080,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
history_df.to_sql(f'{self.symbol}_candle_history',conn,if_exists='append')

In [ ]:
self.historical_info[-1]

In [ ]:
for h in self.historical_info:
    insert_sql = f'INSERT INTO {self.symbol}_trade_history {h}' 
    conn.execute(insert_sql)

In [ ]:
client.klines("BNBUSDT", "1h", limit=1)

In [ ]:
self._get_ticker()

In [ ]:
data_list=client.klines("BNBUSDT", "1h", limit=10)
columns=['date_open','open','high','low','close','volume','date_close','QA_volume','N_trades','BA_volume','BQ_volume','unused']
data=pd.DataFrame(data_list,columns=columns)
data['date_close']=data['date_close'].apply(pd.to_datetime,unit='ms')
data['date_open']=data['date_open'].apply(pd.to_datetime,unit='ms')

# data_list

In [ ]:
data